# Analyse LiDAR et Estimation du Stock de Carbone Végétal

Ce notebook permet la visualisation des données LiDAR et l'évaluation du stock de carbone végétal à partir des modèles de hauteur de canopée.

## Installation des packages nécessaires

```bash
conda create -n geo python
conda activate geo
conda install -c conda-forge mamba
mamba install -c conda-forge pygis
pip install laspy[lazrs] scikit-learn matplotlib scipy
```

Pour Google Colab, décommentez la ligne suivante:

In [ ]:
# !pip install geemap[lidar] scikit-learn matplotlib scipy

## Import des bibliothèques

In [ ]:
import os
import numpy as np
import geemap
import whitebox
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
from scipy import ndimage
from sklearn.cluster import KMeans
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact

In [ ]:
import panel as pn
import threading

## Configuration de whitebox

In [ ]:
wbt = whitebox.WhiteboxTools()
wbt.set_working_dir(os.getcwd())
wbt.set_verbose_mode(False)

## Téléchargement des données d'exemple

In [ ]:
url = "https://github.com/giswqs/data/raw/main/lidar/madison.laz"
if not os.path.exists("madison.laz"):
    geemap.download_file(url)

## Lecture et prétraitement des données LiDAR

In [ ]:
# Lecture du fichier LAZ
laz = geemap.read_lidar("madison.laz")
print(f"Version du fichier: {str(laz.header.version)}")

# Conversion vers une version plus récente si nécessaire
las = geemap.convert_lidar(laz, file_version="1.4")
print(f"Nouvelle version: {str(las.header.version)}")

# Sauvegarde du fichier LAS
geemap.write_lidar(las, "madison.las")

## Analyse d'histogramme et visualisation interactive

In [ ]:
# Création d'un histogramme interactif
wbt.lidar_histogram("madison.las", "histogram.html")

# Affichage du fichier HTML
from IPython.display import IFrame
IFrame(src="histogram.html", width=800, height=600)

In [ ]:
# bonne façon de fixer la taille
viewer = geemap.view_lidar(
    "madison.las",
    backend="pyvista",
    cmap="terrain",
    window_size=(1280, 720),   # au lieu de width, height
)

In [ ]:
import laspy
from pathlib import Path

las_path = Path("madison.las")

with laspy.open(las_path) as fh:
    hdr = fh.header

    print("Nombre total de points :", hdr.point_count)
    print("Format de points (ID)  :", hdr.point_format.id)
    print("Système de coordonnées :", hdr.parse_crs())
    print("Emprise (xmin, xmax, ymin, ymax, zmin, zmax) :",
          (*hdr.mins, *hdr.maxs))              # tuple de 6 valeurs

    # ------------------------------------------------------------------
    # 1) Lister proprement les dimensions
    dims = list(hdr.point_format.dimension_names)
    print("Dimensions standards + extra :", dims)

    # 2) Afficher le détail pour chaque dimension
    xyz_scales  = dict(zip(['X', 'Y', 'Z'], hdr.scales))
    xyz_offsets = dict(zip(['X', 'Y', 'Z'], hdr.offsets))

    for d in hdr.point_format.dimensions:
        name  = d.name
        dtype = d.dtype

        # Échelle / offset uniquement pour X, Y, Z
        scale  = xyz_scales.get(name)
        offset = xyz_offsets.get(name)

        print(f"{name:15s}  dtype={dtype}  "
              f"scale={scale if scale is not None else '—'}  "
              f"offset={offset if offset is not None else '—'}")


In [ ]:
import laspy
import numpy as np
from pathlib import Path

las_path = Path("madison.las")

with laspy.open(las_path) as fh:
    las = fh.read()               # charge tout en mémoire
    hdr = las.header

# ➜ mise à l’échelle (LAS stocke X-Y-Z en entiers)
x = las.X * hdr.scales[0] + hdr.offsets[0]
y = las.Y * hdr.scales[1] + hdr.offsets[1]
z = las.Z * hdr.scales[2] + hdr.offsets[2]

points = np.column_stack((x, y, z))         # (N, 3) float64
intensity = las.intensity                   # ou classification, etc.

# ─────────────────────────────——
# Optionnel : échantillonnage aléatoire pour alléger le rendu
N = points.shape[0]
target = 2_000_000          # vise ~2 millions de points
if N > target:
    idx = np.random.choice(N, target, replace=False)
    points = points[idx]
    intensity = intensity[idx]
    print(f"Nuage réduit : {target:,} pts")

In [ ]:
import pyvista as pv

cloud = pv.PolyData(points)
cloud["intensity"] = intensity   # scalaires pour la couleur

p = pv.Plotter(notebook=True)
p.add_points(
    cloud,
    scalars="intensity",
    cmap="viridis",
    point_size=1,
    render_points_as_spheres=True,
)
p.show(jupyter_backend="pythreejs")   # backend WebGL dans le notebook


In [ ]:
# après avoir construit `points` et `intensity`
limit = 500_000          # cible (à ajuster)
if points.shape[0] > limit:
    step = points.shape[0] // limit
    points     = points[::step]
    intensity  = intensity[::step]
    print(f"Nuage échantillonné tous les {step} points → {len(points):,} pts")

In [ ]:
import pandas as pd
import plotly.express as px

df = pd.DataFrame(points, columns=["x", "y", "z"])
df["intensity"] = intensity

fig = px.scatter_3d(
    df,
    x="x", y="y", z="z",
    color="intensity",
    color_continuous_scale="viridis",
    size_max=1,                 # point_size dans Plotly Express
)
fig.update_traces(marker=dict(size=1))
fig.update_layout(scene_aspectmode="data")  # proportions réelles
fig.show()


In [ ]:
import laspy, geemap, os

in_las = "madison.las"

# ► Vérifier l’entête
las = laspy.read(in_las)
print(las.header.point_count)       # 86 423 880
print(las.header.version)           # 1.2 ?
print(las.header.point_format.id)   # 3

# ► Ajouter / corriger le CRS si besoin
if las.header.epsg != 32633:
    las.header.epsg = 32633         # WGS-84 / UTM 33 N
    las.write("madison.las")


In [ ]:
# Visualisation 3D des données LiDAR
geemap.view_lidar("madison.las")

## Nettoyage des données - Suppression des valeurs aberrantes

In [ ]:
# Définir une plage d'élévation valide pour supprimer les valeurs aberrantes
min_z = 0
max_z = 450

wbt.lidar_elevation_slice("madison.las", "madison.las_rm.las", minz=min_z, maxz=max_z)
print(f"Valeurs aberrantes supprimées (z < {min_z} ou z > {max_z})")

# Visualisation après nettoyage
geemap.view_lidar("madison.las_rm.las", cmap="terrain")

## Création des modèles numériques (DSM, DEM, CHM)

In [ ]:
# Fonction pour créer tous les modèles numériques nécessaires
def generer_modeles_numeriques(resolution=1.0, filtre_dem=25, pente_dem=15.0):
    """Génère les modèles numériques DSM, DEM et CHM à partir des données LiDAR nettoyées."""
    # Création du DSM (Digital Surface Model - Modèle Numérique de Surface)
    wbt.lidar_digital_surface_model(
        "madison_rm.las", "dsm.tif", resolution=resolution, minz=min_z, maxz=max_z
    )
    
    # Ajout du système de coordonnées (EPSG:2255 - NAD83/Wisconsin South)
    geemap.add_crs("dsm.tif", epsg=2255)
    
    # Création du DEM (Digital Elevation Model - Modèle Numérique de Terrain)
    wbt.remove_off_terrain_objects("dsm.tif", "dem.tif", filter=filtre_dem, slope=pente_dem)
    
    # Création du CHM (Canopy Height Model - Modèle de Hauteur de Canopée)
    wbt.subtract("dsm.tif", "dem.tif", "chm.tif")
    
    print("Modèles générés avec succès:")
    print(f"- DSM: résolution {resolution}m")
    print(f"- DEM: filtre {filtre_dem}m, pente {pente_dem}°")
    print("- CHM: différence entre DSM et DEM")
    
    # Application d'un filtre médian au CHM pour lisser les données
    with rasterio.open("chm.tif") as src:
        chm_data = src.read(1)
        profile = src.profile
        
    # Remplacement des valeurs négatives par zéro
    chm_data[chm_data < 0] = 0
    
    # Application d'un filtre médian
    chm_filtered = ndimage.median_filter(chm_data, size=3)
    
    # Sauvegarde du CHM filtré
    with rasterio.open("chm_filtered.tif", 'w', **profile) as dst:
        dst.write(chm_filtered, 1)
    
    print("- CHM filtré: filtre médian appliqué")
    
    return "dsm.tif", "dem.tif", "chm_filtered.tif"

# Génération des modèles avec paramètres par défaut
dsm_path, dem_path, chm_path = generer_modeles_numeriques()

## Interface de visualisation interactive des modèles

In [ ]:
# Fonction pour afficher un modèle spécifique avec une échelle de couleur
def afficher_modele(modele, colormap="terrain"):
    """Affiche un modèle raster avec l'échelle de couleur spécifiée."""
    plt.figure(figsize=(12, 8))
    with rasterio.open(modele) as src:
        data = src.read(1)
        im = plt.imshow(data, cmap=colormap)
        plt.title(f"Visualisation de {os.path.basename(modele)}")
        cbar = plt.colorbar(im)
        cbar.set_label("Élévation (m)")
    plt.show()

# Widget interactif pour visualiser les différents modèles
@interact(modele=["dsm.tif", "dem.tif", "chm_filtered.tif"], colormap=["terrain", "viridis", "plasma", "inferno", "gist_earth", "rainbow"])
def visualiser_modele(modele, colormap):
    afficher_modele(modele, colormap)

## Visualisation sur carte interactive

In [ ]:
import localtileserver
#print(localtileserver.Report())

In [ ]:
import werkzeug
print(werkzeug.__version__)  # doit être 2.2.3

import localtileserver
print("localtileserver OK")


In [ ]:
from localtileserver import get_or_create_tile_client
from ipyleaflet import TileLayer

def safe_add_raster(m, source, colormap="terrain", layer_name="Raster"):
    """Ajoute un raster local à geemap sans dépendre de méthodes internes instables."""
    tile_client = get_or_create_tile_client(source)
    if isinstance(tile_client, tuple):  # corriger si un tuple est retourné
        tile_client = tile_client[0]
    
    tile_url = tile_client.get_tile_url()
    
    tile_layer = TileLayer(
        url=tile_url,
        name=layer_name,
        opacity=1.0,
        visible=True,
    )
    m.add_layer(tile_layer)
    print(f"✅ {layer_name} ajouté avec succès.")

In [ ]:
import ee
ee.Authenticate()
ee.Initialize(project='ee-falama2018')

m = geemap.Map(ee_initialize=False)

safe_add_raster(m, "dsm.tif", colormap="terrain", layer_name="DSM")
safe_add_raster(m, "dem.tif", colormap="terrain", layer_name="DEM")
safe_add_raster(m, "chm_filtered.tif", colormap="viridis", layer_name="CHM")

m

In [ ]:
# Création d'une carte interactive avec tous les modèles
import ee
ee.Authenticate()
ee.Initialize(project='ee-falama2018')

m = geemap.Map()
#m.add_raster('dsm.tif')
m.add_raster("dsm.tif", colormap="terrain", layer_name="DSM")
m.add_raster("dem.tif", colormap="terrain", layer_name="DEM")
m.add_raster("chm_filtered.tif", colormap="viridis", layer_name="CHM")
m

## Classification de la végétation à partir du CHM

In [ ]:
def classifier_vegetation(chm_path, n_classes=5):
    """Classifie la végétation en fonction de la hauteur de canopée."""
    with rasterio.open(chm_path) as src:
        chm_data = src.read(1)
        profile = src.profile
    
    # Masque pour exclure les zones sans végétation (CHM <= 0.5m)
    mask = chm_data > 0.5
    
    # Extraction des hauteurs de végétation
    veg_heights = chm_data[mask]
    
    if len(veg_heights) == 0:
        print("Pas de végétation détectée avec une hauteur > 0.5m")
        return None, None
    
    # Création de classes basées sur les hauteurs naturelles
    # Nous pouvons utiliser KMeans ou des seuils prédéfinis
    
    # Option 1: KMeans pour une classification automatique
    veg_heights_2d = veg_heights.reshape(-1, 1)
    kmeans = KMeans(n_clusters=n_classes, random_state=42).fit(veg_heights_2d)
    classes = kmeans.labels_
    
    # Créer une carte de classification
    classe_map = np.zeros_like(chm_data)
    classe_map[mask] = classes + 1  # +1 pour réserver 0 pour 'pas de végétation'
    
    # Sauvegarde de la carte de classification
    profile.update(
        dtype=rasterio.uint8,
        nodata=0  # ✅ correction ici
    )
    with rasterio.open("vegetation_classes.tif", 'w', **profile) as dst:
        dst.write(classe_map.astype(rasterio.uint8), 1)
    
    # Analyse des centres de cluster pour déterminer les types de végétation
    centers = kmeans.cluster_centers_.flatten()
    sorted_indices = np.argsort(centers)
    
    # Définition des types de végétation en fonction de la hauteur
    types_vegetation = {
        1: "Pas de végétation (< 0.5m)",
    }
    
    for i, idx in enumerate(sorted_indices):
        hauteur = centers[idx]
        classe = idx + 1  # +1 car nous avons réservé 0 pour 'pas de végétation'
        
        if hauteur < 1:
            type_veg = "Végétation basse (herbe, arbustes)"
        elif hauteur < 5:
            type_veg = "Arbustes / Petits arbres"
        elif hauteur < 15:
            type_veg = "Arbres moyens"
        else:
            type_veg = "Grands arbres"
            
        types_vegetation[classe + 1] = f"{type_veg} (∼{hauteur:.1f}m)"
    
    print("Classification de la végétation terminée")
    print("Classes de végétation:")
    for classe, desc in types_vegetation.items():
        print(f"  Classe {classe}: {desc}")
    
    return "vegetation_classes.tif", types_vegetation

# Classification de la végétation
veg_classes_path, types_veg = classifier_vegetation(chm_path)

## Visualisation de la classification de végétation

In [ ]:
if veg_classes_path:
    # Création d'une colormap personnalisée pour les classes de végétation
    colors = ['#FFFFFF', '#C2DF23', '#1E6C0B', '#1A4314', '#0B2307']
    n_classes = len(types_veg)
    
    # Affichage de la carte de classification
    plt.figure(figsize=(12, 8))
    with rasterio.open(veg_classes_path) as src:
        veg_data = src.read(1)
        plt.imshow(veg_data, cmap=plt.cm.colors.ListedColormap(colors[:n_classes]))
        plt.colorbar(ticks=range(n_classes), label="Classes de végétation")
        plt.title("Classification de la végétation")
        
        # Ajouter une légende personnalisée
        patches = [plt.Rectangle((0, 0), 1, 1, fc=colors[i]) for i in range(n_classes)]
        plt.legend(patches, [desc for _, desc in types_veg.items()], 
                 loc='lower right', frameon=True)
    plt.show()
    
    # Ajouter la classification à la carte interactive
    #m.add_raster(veg_classes_path, colormap=colors[:n_classes], layer_name="Classification de végétation")
    #m

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import rasterio

# Exemple : types_veg = {1: "Herbacée", 2: "Arbustive", ...}
n_classes = len(types_veg)

# Génère une palette qualitative avec autant de couleurs que de classes
cmap_base = mpl.cm.get_cmap('viridis', n_classes)       # ou 'tab20', 'Set3', etc.
colors = [mpl.colors.to_hex(cmap_base(i)) for i in range(cmap_base.N)]

# --- Affichage matriciel ---
plt.figure(figsize=(12, 8))
with rasterio.open(veg_classes_path) as src:
    veg_data = src.read(1)
    plt.imshow(veg_data, cmap=mpl.colors.ListedColormap(colors))
    plt.colorbar(ticks=range(1, n_classes + 1), label="Classes de végétation")
    plt.title("Classification de la végétation")

    patches = [plt.Rectangle((0, 0), 1, 1, fc=colors[i])
               for i in range(n_classes)]
    plt.legend(
        patches,
        [desc for _, desc in types_veg.items()],
        loc='lower right',
        frameon=True
    )
plt.show()

# --- Carte interactive (geemap / folium) ---
#m.add_raster(
#    veg_classes_path,
#    colormap=colors,
#    layer_name="Classification de la végétation"
#)
#m

## Estimation du stock de carbone végétal

In [ ]:
def estimer_carbone(chm_path, pixel_size=1.0):
    """Estime le stock de carbone végétal à partir du CHM.
    
    Cette fonction utilise des équations allométriques simplifiées pour estimer:
    1. Le volume de végétation
    2. La biomasse aérienne (AGB - Above-Ground Biomass)
    3. Le stock de carbone (généralement ~50% de la biomasse sèche)
    """
    with rasterio.open(chm_path) as src:
        chm_data = src.read(1)
        profile = src.profile
    
    # Surface de pixel en m²
    pixel_area = pixel_size * pixel_size
    
    # 1. Calcul du volume (m³) = hauteur × surface
    volume = chm_data * pixel_area
    
    # 2. Conversion du volume en biomasse (kg) avec facteurs de densité spécifiques
    # Nous utilisons différents facteurs selon la hauteur de végétation
    biomasse = np.zeros_like(volume)
    
    # Facteurs de densité approximatifs (kg/m³)
    # Ces valeurs sont approximatives et devraient être adaptées au type de forêt
    biomasse[(chm_data > 0) & (chm_data <= 1)] = volume[(chm_data > 0) & (chm_data <= 1)] * 2.0  # Végétation basse
    biomasse[(chm_data > 1) & (chm_data <= 5)] = volume[(chm_data > 1) & (chm_data <= 5)] * 5.0  # Arbustes
    biomasse[(chm_data > 5) & (chm_data <= 15)] = volume[(chm_data > 5) & (chm_data <= 15)] * 10.0  # Arbres moyens
    biomasse[chm_data > 15] = volume[chm_data > 15] * 15.0  # Grands arbres
    
    # 3. Conversion de la biomasse en carbone (kg) - généralement ~50% de la biomasse sèche
    carbone = biomasse * 0.5
    
    # Sauvegarde des rasters de résultats
    with rasterio.open("biomasse.tif", 'w', **profile) as dst:
        dst.write(biomasse.astype(rasterio.float32), 1)
    
    with rasterio.open("carbone.tif", 'w', **profile) as dst:
        dst.write(carbone.astype(rasterio.float32), 1)
    
    # Calcul des statistiques totales
    total_volume = np.sum(volume)
    total_biomasse = np.sum(biomasse)
    total_carbone = np.sum(carbone)
    
    # Conversion en tonnes pour les rapports
    total_biomasse_tonnes = total_biomasse / 1000
    total_carbone_tonnes = total_carbone / 1000
    
    # Calcul des statistiques par classe de hauteur
    classes_hauteur = {
        "Végétation basse (< 1m)": (0, 1),
        "Arbustes (1-5m)": (1, 5),
        "Arbres moyens (5-15m)": (5, 15),
        "Grands arbres (>15m)": (15, float('inf'))
    }
    
    stats_classes = {}
    for classe, (min_h, max_h) in classes_hauteur.items():
        mask = (chm_data > min_h) & (chm_data <= max_h)
        stats_classes[classe] = {
            "surface_ha": np.sum(mask) * pixel_area / 10000,  # en hectares
            "volume_m3": np.sum(volume[mask]),
            "biomasse_tonnes": np.sum(biomasse[mask]) / 1000,
            "carbone_tonnes": np.sum(carbone[mask]) / 1000
        }
    
    print("\n=== Estimation du stock de carbone ===\n")
    print(f"Surface totale analysée: {np.sum(chm_data > 0) * pixel_area / 10000:.2f} hectares")
    print(f"Volume total de végétation: {total_volume:.2f} m³")
    print(f"Biomasse totale: {total_biomasse_tonnes:.2f} tonnes")
    print(f"Stock de carbone total: {total_carbone_tonnes:.2f} tonnes CO₂ équivalent")
    
    print("\nRépartition par classe de hauteur:")
    for classe, stats in stats_classes.items():
        print(f"\n{classe}:")
        print(f"  Surface: {stats['surface_ha']:.2f} ha")
        print(f"  Volume: {stats['volume_m3']:.2f} m³")
        print(f"  Biomasse: {stats['biomasse_tonnes']:.2f} tonnes")
        print(f"  Carbone: {stats['carbone_tonnes']:.2f} tonnes CO₂ eq.")
    
    return "biomasse.tif", "carbone.tif", stats_classes

# Estimation du stock de carbone
biomasse_path, carbone_path, stats_carbone = estimer_carbone(chm_path)

## Visualisation des résultats de l'estimation du carbone

In [ ]:
# Visualisation de la biomasse et du carbone
plt.figure(figsize=(15, 6))

plt.subplot(1, 2, 1)
with rasterio.open(biomasse_path) as src:
    biomasse_data = src.read(1)
    im = plt.imshow(biomasse_data, cmap='YlGn')
    plt.colorbar(im, label="Biomasse (kg/pixel)")
    plt.title("Estimation de la biomasse")

plt.subplot(1, 2, 2)
with rasterio.open(carbone_path) as src:
    carbone_data = src.read(1)
    im = plt.imshow(carbone_data, cmap='YlGn')
    plt.colorbar(im, label="Carbone (kg/pixel)")
    plt.title("Estimation du stock de carbone")

plt.tight_layout()
plt.show()

# Ajouter les couches à la carte interactive
#m.add_raster(biomasse_path, colormap="YlGn", layer_name="Biomasse (kg/pixel)")
#m.add_raster(carbone_path, colormap="YlGn", layer_name="Carbone (kg/pixel)")
#m

## Graphiques de répartition du carbone

In [ ]:
# Création d'un DataFrame à partir des statistiques de carbone
data = []
for classe, stats in stats_carbone.items():
    data.append({
        "Classe": classe,
        "Surface (ha)": stats["surface_ha"],
        "Volume (m³)": stats["volume_m3"],
        "Biomasse (t)": stats["biomasse_tonnes"],
        "Carbone (t CO₂ eq.)": stats["carbone_tonnes"]
    })

df = pd.DataFrame(data)
print(df)

# Graphiques de répartition
plt.figure(figsize=(18, 10))

# 1. Répartition de la surface par classe
plt.subplot(2, 2, 1)
plt.pie(df["Surface (ha)"], labels=df["Classe"], autopct='%1.1f%%', startangle=90)
plt.title("Répartition de la surface par classe de végétation")

# 2. Répartition du volume par classe
plt.subplot(2, 2, 2)
plt.pie(df["Volume (m³)"], labels=df["Classe"], autopct='%1.1f%%', startangle=90)
plt.title("Répartition du volume par classe de végétation")

# 3. Répartition de la biomasse par classe
plt.subplot(2, 2, 3)
plt.pie(df["Biomasse (t)"], labels=df["Classe"], autopct='%1.1f%%', startangle=90)
plt.title("Répartition de la biomasse par classe de végétation")

# 4. Répartition du carbone par classe
plt.subplot(2, 2, 4)
plt.pie(df["Carbone (t CO₂ eq.)"], labels=df["Classe"], autopct='%1.1f%%', startangle=90)
plt.title("Répartition du stock de carbone par classe de végétation")

plt.tight_layout()
plt.show()

# Graphique comparatif en barres
plt.figure(figsize=(14, 8))
df.plot(x="Classe", y=["Carbone (t CO₂ eq.)", "Biomasse (t)"], kind="bar")
plt.title("Comparaison de la biomasse et du stock de carbone par classe de végétation")
plt.ylabel("Tonnes")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

## Rapport d'évaluation du stock de carbone végétal

In [ ]:
# Création d'un rapport d'évaluation du stock de carbone
from IPython.display import Markdown

with rasterio.open(chm_path) as src:
    chm_data = src.read(1)
    pixel_area = 1.0 * 1.0  # en m²
    surface_totale_ha = np.sum(chm_data > 0) * pixel_area / 10000

rapport = f"""
# Rapport d'évaluation du stock de carbone végétal

## Résumé

L'analyse des données LiDAR a permis d'évaluer la distribution spatiale de la végétation et d'estimer le stock de carbone végétal sur la zone d'étude.

- **Surface totale analysée**: {surface_totale_ha:.2f} hectares
- **Stock de carbone total**: {df['Carbone (t CO₂ eq.)'].sum():.2f} tonnes CO₂ équivalent
- **Densité moyenne de carbone**: {df['Carbone (t CO₂ eq.)'].sum() / surface_totale_ha:.2f} tonnes CO₂ eq./ha

## Répartition par classe de végétation

| Classe de végétation | Surface (ha) | Volume (m³) | Biomasse (t) | Carbone (t CO₂ eq.) | Densité (t CO₂/ha) |
|---------------------|--------------|-------------|--------------|---------------------|--------------------|
"""

for _, row in df.iterrows():
    densite = row["Carbone (t CO₂ eq.)"] / row["Surface (ha)"] if row["Surface (ha)"] > 0 else 0
    rapport += f"| {row['Classe']} | {row['Surface (ha)']:.2f} | {row['Volume (m³)']:.2f} | {row['Biomasse (t)']:.2f} | {row['Carbone (t CO₂ eq.)']:.2f} | {densite:.2f} |\n"

rapport += f"""

## Méthodologie

1. **Prétraitement des données LiDAR**:
   - Suppression des valeurs aberrantes
   - Génération des modèles numériques (DSM, DEM, CHM)

2. **Classification de la végétation**:
   - Segmentation basée sur la hauteur de canopée
   - Identification de {len(df)} classes de végétation

3. **Estimation du stock de carbone**:
   - Calcul du volume de végétation à partir du CHM
   - Application de facteurs de conversion biomasse/carbone spécifiques à chaque classe
   - Estimation du stock de carbone en équivalent CO₂

## Remarques

- Les estimations sont basées sur des équations allométriques générales et pourraient être affinées avec des données spécifiques aux espèces présentes sur le site.
- La précision des estimations dépend de la qualité des données LiDAR et de la résolution spatiale.
- Pour une analyse plus précise, il est recommandé de compléter cette évaluation par des mesures de terrain.
"""

# Affichage du rapport
Markdown(rapport)

## Téléchargement des résultats

In [ ]:
# Création d'un zip avec tous les fichiers résultats
import zipfile

files_to_zip = [
    "madison_rm.las",
    "dsm.tif",
    "dem.tif",
    "chm_filtered.tif",
    "vegetation_classes.tif",
    "biomasse.tif",
    "carbone.tif",
    "histogram.html"
]

# Sauvegarde du rapport en Markdown
with open("rapport_carbone.md", "w") as f:
    f.write(rapport)

files_to_zip.append("rapport_carbone.md")

# Création du fichier zip
with zipfile.ZipFile("resultats_lidar_carbone.zip", "w") as zipf:
    for file in files_to_zip:
        if os.path.exists(file):
            zipf.write(file)
            print(f"Ajout de {file} au fichier zip")

print(f"\nFichier zip créé: resultats_lidar_carbone.zip")

# Téléchargement du fichier zip (dans Colab)
try:
    from google.colab import files
    files.download("resultats_lidar_carbone.zip")
    print("Téléchargement lancé...")
except:
    print("Vous pouvez télécharger le fichier zip manuellement depuis votre répertoire de travail.")

In [ ]:
import os
import zipfile

# ------------------------------------------------------------------
# 1.  Liste des fichiers à inclure
# ------------------------------------------------------------------
files_to_zip = [
    "madison_rm.las",
    "dsm.tif",
    "dem.tif",
    "chm_filtered.tif",
    "vegetation_classes.tif",
    "biomasse.tif",
    "carbone.tif",
    "histogram.html",
]

# ------------------------------------------------------------------
# 2.  Sauvegarde du rapport Markdown avec encodage UTF-8
# ------------------------------------------------------------------
with open("rapport_carbone.md", "w", encoding="utf-8") as f:
    f.write(rapport)

files_to_zip.append("rapport_carbone.md")

# ------------------------------------------------------------------
# 3.  Création de l’archive ZIP
# ------------------------------------------------------------------
zip_name = "resultats_lidar_carbone.zip"
with zipfile.ZipFile(zip_name, "w", compression=zipfile.ZIP_DEFLATED) as zipf:
    for file in files_to_zip:
        if os.path.isfile(file):
            zipf.write(file)
            print(f"➕  Ajout de {file}")
        else:
            print(f"⚠️  Fichier introuvable : {file}")

print(f"\n✅  Fichier ZIP créé : {zip_name}")

# ------------------------------------------------------------------
# 4.  Téléchargement (si vous êtes dans Colab)
# ------------------------------------------------------------------
try:
    from google.colab import files
    files.download(zip_name)
    print("Téléchargement lancé…")
except ImportError:
    # Cas : Jupyter local, VS Code, etc.
    from IPython.display import FileLink, display
    display(FileLink(zip_name))
    print("Cliquez sur le lien ci-dessus pour télécharger l’archive.")